In [48]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [49]:
# Define constants
CLASS_NAMES = ['CaS', 'CoS', 'Gum', 'MC', 'OC', 'OLP', 'OT']
IMG_SIZE = 224
BATCH_SIZE = 32

# Set up path
base_dir = '/kaggle/input/teeth-dataset/Teeth_Dataset'
train_dir = os.path.join(base_dir, 'Training')
valid_dir = os.path.join(base_dir, 'Validation')
test_dir = os.path.join(base_dir, 'Testing')

# Verify paths and data
print("Available directories:")
print(f"Training: {os.path.exists(train_dir)}")
print(f"Validation: {os.path.exists(valid_dir)}")
print(f"Testing: {os.path.exists(test_dir)}")

Available directories:
Training: True
Validation: True
Testing: True


In [50]:
# Count images in each directory
def count_images(directory):
    counts = {}
    for class_name in CLASS_NAMES:
        class_dir = os.path.join(directory, class_name)
        if os.path.exists(class_dir):
            counts[class_name] = len(os.listdir(class_dir))
    return counts

print("\nImage counts:")
print("Training:", count_images(train_dir))
print("Validation:", count_images(valid_dir))
print("Testing:", count_images(test_dir))


Image counts:
Training: {'CaS': 480, 'CoS': 450, 'Gum': 360, 'MC': 540, 'OC': 324, 'OLP': 540, 'OT': 393}
Validation: {'CaS': 160, 'CoS': 149, 'Gum': 120, 'MC': 180, 'OC': 108, 'OLP': 180, 'OT': 131}
Testing: {'CaS': 160, 'CoS': 149, 'Gum': 120, 'MC': 180, 'OC': 108, 'OLP': 180, 'OT': 131}


In [51]:
# Create data generators
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASS_NAMES,
    shuffle=True
)

validation_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASS_NAMES,
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASS_NAMES,
    shuffle=False
)

Found 3087 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.


In [52]:
# Create base model
base_model = EfficientNetB3(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# Create model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, 
                activation='relu',
                kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(len(CLASS_NAMES), activation='softmax')
])

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy',
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [53]:
# Train the model
initial_history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=40,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=8,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5
        )
    ]
)

Epoch 1/40


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


97/97 ━━━━━━━━━━━━━━━━━━━━ 230s 1s/step - accuracy: 0.5180 - loss: 3.8489 - precision: 0.5921 - recall: 0.4456 - val_accuracy: 0.1819 - val_loss: 3.8538 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/40
97/97 ━━━━━━━━━━━━━━━━━━━━ 37s 357ms/step - accuracy: 0.8928 - loss: 1.9946 - precision: 0.9195 - recall: 0.8740 - val_accuracy: 0.1556 - val_loss: 3.1893 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/40
97/97 ━━━━━━━━━━━━━━━━━━━━ 37s 351ms/step - accuracy: 0.9066 - loss: 1.3313 - precision: 0.9210 - recall: 0.8902 - val_accuracy: 0.1245 - val_loss: 2.8816 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 4/40
97/97 ━━━━━━━━━━━━━━━━━━━━ 37s 355ms/step - accuracy: 0.9458 - loss: 0.8080 - precision: 0.9538 - recall: 0.9376 - val_accuracy: 0.2111 - val_loss: 2.7352 - val_precision: 0.3105 - val_recall: 0.0574 - learning_rate: 0.0010
Epoch 5/40
97/97 ━━━━━━━━━━━━━━━━━━━━ 38s 364ms/s

In [54]:
# Prepare model for fine-tuning
base_model = model.layers[0]
base_model.trainable = True

# Freeze all layers except the last 15
for layer in base_model.layers[:-15]:
    layer.trainable = False

# Recompile model with lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy',
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=6,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=4,
            min_lr=1e-7
        )
    ]
)

Epoch 1/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 94s 600ms/step - accuracy: 0.9953 - loss: 0.0181 - precision: 0.9973 - recall: 0.9953 - val_accuracy: 0.9961 - val_loss: 0.0211 - val_precision: 0.9961 - val_recall: 0.9961 - learning_rate: 1.0000e-05
Epoch 2/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 338ms/step - accuracy: 0.9996 - loss: 0.0137 - precision: 0.9996 - recall: 0.9996 - val_accuracy: 0.9961 - val_loss: 0.0208 - val_precision: 0.9961 - val_recall: 0.9961 - learning_rate: 1.0000e-05
Epoch 3/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 332ms/step - accuracy: 0.9983 - loss: 0.0138 - precision: 0.9983 - recall: 0.9982 - val_accuracy: 0.9971 - val_loss: 0.0205 - val_precision: 0.9971 - val_recall: 0.9971 - learning_rate: 1.0000e-05
Epoch 4/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 336ms/step - accuracy: 0.9987 - loss: 0.0164 - precision: 0.9987 - recall: 0.9987 - val_accuracy: 0.9971 - val_loss: 0.0202 - val_precision: 0.9971 - val_recall: 0.9971 - learning_rate: 1.0000e-05
Epoch 5/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 36s 34

In [55]:
# Evaluate model
test_results = model.evaluate(test_generator)
print("\nTest Results:")
print(f"Test Loss: {test_results[0]:.4f}")
print(f"Test Accuracy: {test_results[1]:.4f}")
print(f"Test Precision: {test_results[2]:.4f}")
print(f"Test Recall: {test_results[3]:.4f}")

# Save model
model.save('teeth_classification_model.h5')

33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 190ms/step - accuracy: 1.0000 - loss: 0.0054 - precision: 1.0000 - recall: 1.0000

Test Results:
Test Loss: 0.0063
Test Accuracy: 1.0000
Test Precision: 1.0000
Test Recall: 1.0000


In [56]:
!pip install gradio

In [57]:
import os
print("Files in working directory:", os.listdir("/kaggle/working/"))


Files in working directory: ['app.py', 'teeth_classification_model.h5', '.virtual_documents']


In [58]:

import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the trained model
model = tf.keras.models.load_model("/kaggle/working/teeth_classification_model.h5")

CLASS_NAMES = ['CaS', 'CoS', 'Gum', 'MC', 'OC', 'OLP', 'OT']

def predict_teeth_disease(img):
    img = img.resize((224, 224))  # Resize image
    img_array = np.array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions for model
    prediction = model.predict(img_array)[0]
    return {CLASS_NAMES[i]: float(prediction[i]) for i in range(len(CLASS_NAMES))}

# Launch the Gradio interface
interface = gr.Interface(
    fn=predict_teeth_disease,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(),
    title="Teeth Disease Classification",
    description="Upload an image to classify different types of teeth conditions."
)

interface.launch(share=True)  # Generates a public URL for access


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://dc93da07246ac028ca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
